### train, test 데이터셋 agg 처리 후 parquet로 저정하기

In [1]:
import gc

import pandas as pd

In [2]:
train = pd.read_parquet('./data/train.parquet')
train_labels = pd.read_csv('./data/train_labels.csv')

train.shape, train_labels.shape

((5531451, 190), (458913, 2))

In [3]:
train = train.merge(train_labels, how='inner', on='customer_ID')
del train_labels
gc.collect()

train.shape

(5531451, 191)

In [4]:
features = train.drop(['customer_ID', 'S_2', 'target'], axis=1).columns.to_list()
categorical_features = [
    'B_30', 'B_38', 'D_63', 'D_64', 'D_66', 'D_68',
    'D_114', 'D_116', 'D_117', 'D_120', 'D_126'
]
num_features = [col for col in features if col not in categorical_features]

In [5]:
train_num_agg = train.groupby('customer_ID')[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]

train_cat_agg = train.groupby('customer_ID')[categorical_features].agg(['count', 'last', 'nunique'])
train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]

train_target = (train.groupby('customer_ID').tail(1).set_index('customer_ID', drop=True).sort_index()['target'])

train = pd.concat([train_num_agg, train_cat_agg, train_target], axis=1)

train.to_parquet('./data/test/train_agg.parquet', engine='pyarrow', compression='gzip')

del train_num_agg, train_cat_agg, train_target
gc.collect()

0

In [6]:
del train
gc.collect()

20

In [7]:
test = pd.read_parquet('./data/test.parquet')
test.shape

(11363762, 190)

In [8]:
test_num_agg = test.groupby('customer_ID')[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]

test_cat_agg = test.groupby('customer_ID')[categorical_features].agg(['count', 'last', 'nunique'])
test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]

test = pd.concat([test_num_agg, test_cat_agg], axis=1)

test.to_parquet('./data/test/test_agg.parquet', engine='pyarrow', compression='gzip')

del test_num_agg, test_cat_agg
gc.collect()

0

In [9]:
del test
gc.collect()

20